In [1]:
%pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import math
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta
from statsmodels.tsa.arima.model import ARIMA
import time


In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [4]:
#Name The Exercise
Exercise = 'squat'

In [14]:
#Opens the camera and record a figure doing an excersice.
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }
time.sleep(5)

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)



            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )

            if Exercise != 'bench':
                # Get coordinates
                R1 = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                R2 = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                R3 = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                #Visibility Extraction
                R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
                

                L1 = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                L2 = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                L3 = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                #Visibility Extraction
                L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
                    
            else:
                # Get coordinates
                R1 = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                R2 = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                R3 = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                #Visibility Extraction
                R_V = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                

                L1 = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                L2 = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                L3 = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                #Visibility Extraction
                L_V = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                    
            if R_V > L_V:
                angle = calculate_angle(R1, R2, R3)
            else:
                angle = calculate_angle(L1, L2, L3)
                
            counter+=1

                        # Visualize angle
            cv2.putText(image, str(angle), 
                            #tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                            tuple(np.multiply(L2, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA

                                )
            if counter == 4:
                counter = 0
                iter += 1

            """ 
            c = 0
            avg_y = 0
            for i in range(11):
                if landmarks[i+11].visibility > 0.9:
                    avg_y += landmarks[i+11].y
                    c += 1
            avg_y / c
            """
            
            avg_y = (landmarks[11].y + landmarks[13].y + landmarks[15].y)/3

            if Exercise == 'deadlift':
                max_0 = 165
                min_0 = 135
            elif Exercise == 'squat':
                max_0 = 160
                min_0 = 110
            else:
                max_0 = 165
                min_0 = 95



            if angle > max_0:
                data["spike/deep"].append(1)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            elif angle < min_0:
                data["spike/deep"].append(-1)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            else:
                data["spike/deep"].append(0)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        
            
            cv2.imshow('Mediapipe Feed', image)
            
            if (cv2.waitKey(10) & 0xFF == ord('q') | (iter == 100)):
                break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.7704496383666992
y: 0.16289830207824707
z: -0.4930172562599182
visibility: 0.9999943971633911
, x: 0.7806857824325562
y: 0.14364638924598694
z: -0.4688335359096527
visibility: 0.9999933242797852
, x: 0.786125123500824
y: 0.14466844499111176
z: -0.46880707144737244
visibility: 0.9999899864196777
, x: 0.7919942736625671
y: 0.1462239921092987
z: -0.46892017126083374
visibility: 0.9999889135360718
, x: 0.7619551420211792
y: 0.14088422060012817
z: -0.47083431482315063
visibility: 0.9999957084655762
, x: 0.7532736659049988
y: 0.13991779088974
z: -0.47090092301368713
visibility: 0.9999947547912598
, x: 0.7446818947792053
y: 0.13946300745010376
z: -0.470869779586792
visibility: 0.9999961853027344
, x: 0.7963799834251404
y: 0.15512362122535706
z: -0.300262451171875
visibility: 0.9999822378158569
, x: 0.7306511998176575
y: 0.14758804440498352
z: -0.307616651058197
visibility: 0.9999971389770508
, x: 0.7776177525520325
y: 0.18689599633216858
z: -0.4275903105735779
visibility: 0.999993801116

In [15]:
sum = 0
for i in range(len(data)-1):
    d1 = datetime.strptime(data["dates"][i], "%H:%M:%S:%f")
    d2 = datetime.strptime(data["dates"][i+1], "%H:%M:%S:%f")
    delta = d2 - d1
    delta_ms = round(delta.microseconds,0) / 1000
    sum = sum + delta_ms
avg_ms = sum // len(data)-1
print(avg_ms)

43.0


In [16]:
for i in range(100):
    #index = 121 - i
    d1 = datetime.strptime(data["dates"][-1], "%H:%M:%S:%f")
    sum = d1 + timedelta(milliseconds=avg_ms)
    data["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
    data["values"].append(0)
    data["spike/deep"].append(0)
    #data["values"].append(data["values"][len(data)-index:][0])
    #data["spike/deep"].append(data["spike/deep"][len(data)-index:][0])

In [17]:
df = pd.DataFrame(data = data)

In [18]:
#To Dataframe
df = pd.DataFrame(data = data)

#df_raw = df_raw.drop(index=range(28))

#without Outliers
#df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
#            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [19]:
#Export to CSV
df.to_csv("Squats_Unfiltered.csv" , index = False)

In [20]:
#Plot the graph
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [21]:
#See seasonal components
res = seasonal_decompose(df["values"][:len(df)-100],model = "additive",period=21)
res.plot();

In [15]:
#Autoarima without causal factors
############## NO SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], m=21,trace=True, seasonal=True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[21] intercept   : AIC=inf, Time=6.90 sec
 ARIMA(0,0,0)(0,0,0)[21] intercept   : AIC=-678.462, Time=0.21 sec
 ARIMA(1,0,0)(1,0,0)[21] intercept   : AIC=inf, Time=2.64 sec
 ARIMA(0,0,1)(0,0,1)[21] intercept   : AIC=-2223.965, Time=1.84 sec
 ARIMA(0,0,0)(0,0,0)[21]             : AIC=2158.600, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[21] intercept   : AIC=-2038.584, Time=0.37 sec
 ARIMA(0,0,1)(1,0,1)[21] intercept   : AIC=inf, Time=3.55 sec
 ARIMA(0,0,1)(0,0,2)[21] intercept   : AIC=inf, Time=13.84 sec
 ARIMA(0,0,1)(1,0,0)[21] intercept   : AIC=-2041.400, Time=2.24 sec
 ARIMA(0,0,1)(1,0,2)[21] intercept   : AIC=inf, Time=17.57 sec
 ARIMA(0,0,0)(0,0,1)[21] intercept   : AIC=-875.668, Time=0.60 sec
 ARIMA(1,0,1)(0,0,1)[21] intercept   : AIC=-5413.969, Time=2.09 sec
 ARIMA(1,0,1)(0,0,0)[21] intercept   : AIC=-5392.675, Time=0.47 sec
 ARIMA(1,0,1)(1,0,1)[21] intercept   : AIC=inf, Time=3.31 sec
 ARIMA(1,0,1)(0,0,2)[21] intercept   : AIC

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                 1047
Model:             SARIMAX(2, 0, 1)x(0, 0, [1, 2], 21)   Log Likelihood                2858.956
Date:                                 Sun, 21 May 2023   AIC                          -5703.912
Time:                                         12:36:07   BIC                          -5669.237
Sample:                                              0   HQIC                         -5690.763
                                                - 1047                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0075      0.001     10.550      0.000       0.006       0.009
ar.L1          1.9451      0.011    184.950      0.000       1.925       1.966
ar.L2         -0.9566      0.011    -91.026      0.000      -0.977      -0.936
ma.L1         -0.7415      0.023    -32.481      0.000      -0.786      -0.697
ma.S.L21       0.0866      0.060      1.439      0.150      -0.031       0.205
ma.S.L42      -0.1701      0.049     -3.455      0.001      -0.267      -0.074
sigma2         0.0002   2.82e-06     87.660      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.79   Jarque-Bera (JB):            663727.60
Prob(Q):                              0.38   Prob(JB):                         0.00
Heteroskedasticity (H):               3.44   Skew:                            -6.12
Prob(H) (two-sided):                  0.00   Kurtosis:                       125.74
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
"""for i in range(100):
    index = 100 - i
    if data["values"][len(df)-index:][0] == 0:
       data["values"][len(df)-index:][0] = data["values"][len(df)-index-17:][0]
       print(data["values"][len(df)-index-17:][0])
"""

In [24]:
#Use of sarima without CF
############## NO SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:]
model = SARIMAX(train_df["values"], order=(70, 1, 1))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [37]:
############## NO SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy_no_deeps = 1- rmse(test_df["values"], prediction)
print(accuracy_no_deeps)

0.31595303227378735


In [26]:
#Autoarima with causal factors
############## WITH SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], exogenous=df["spike/deep"], seasonal=True , start_q=0,max_q= 1, trace  =True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-5482.613, Time=0.15 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-678.462, Time=0.15 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-2038.584, Time=0.37 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=2158.600, Time=0.05 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=-5539.710, Time=0.59 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=-5700.186, Time=0.36 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : AIC=-5721.053, Time=0.80 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : AIC=-5705.799, Time=1.63 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=-5701.242, Time=1.32 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=inf, Time=0.34 sec

Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept
Total fit time: 5.990 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1047
Model:               SARIMAX(5, 0, 0)   Log Likelihood                2867.527
Date:                Sun, 21 May 2023   AIC                          -5721.053
Time:                        15:17:59   BIC                          -5686.377
Sample:                             0   HQIC                         -5707.903
                               - 1047                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0131      0.003      4.684      0.000       0.008       0.019
ar.L1          1.2365      0.028     44.815      0.000       1.182       1.291
ar.L2         -0.1783      0.038     -4.732      0.000      -0.252      -0.104
ar.L3          0.2598      0.021     12.285      0.000       0.218       0.301
ar.L4         -0.1905      0.013    -14.229      0.000      -0.217      -0.164
ar.L5         -0.1476      0.009    -17.229      0.000      -0.164      -0.131
sigma2         0.0002   2.35e-06    103.421      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.78   Jarque-Bera (JB):            425758.61
Prob(Q):                              0.38   Prob(JB):                         0.00
Heteroskedasticity (H):               3.14   Skew:                            -4.63
Prob(H) (two-sided):                  0.00   Kurtosis:                       101.36
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [22]:
#Use of sarima with CF
############## WITH SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:] 
model = SARIMAX(endog=train_df["values"], order=(30,1, 0), exog=train_df["spike/deep"])
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end,exog = test_df["spike/deep"]).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [ ]:
############## WITH SPIKES / DEEPS ##################

# Need to make sure to truncate it in the right place
#prediction

In [24]:
############## WITH SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy = 1- rmse(test_df["values"], prediction)
print(accuracy)

0.37624656042425186


In [25]:
# Old Version Of "calculate_decay_rate()"
def calculate_decay_rate(df):
    decay = []
    #average decay rate
    for i in range(1,len(df) -1 ):
        x1 = df["spike/deep"][i]
        x2 = df["spike/deep"][i+1]
        if (x1 == 1) and (x2 == 0):
            x1_index = i
        elif (x1 == 0)  and (x2 == 1):
            decay.append(x1_index/i)
    decay.append(1)
    return decay


In [33]:
#insert empty (only dates) rows to the data
def insert_missing_rows(df,lead):
    for i in range(lead):
        d = datetime.strptime(df["dates"][-1], "%H:%M:%S:%f")
        sum = d + timedelta(seconds=1)
        df["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
        df["values"].append(0)
    return df


In [27]:
#New Verson Of calculate_decay_rate()
#Calculates the decay insert the results to the dictionary 'decay'.
def calculate_decay_rate(df):
    decay = {"dates" : [],
            "values" : []
        }
    try:
        if Exercise != 'deadlift':
            for i in range(1,len(df) -1 ):
                x1 = df["spike/deep"][i]
                x2 = df["spike/deep"][i+1]
                if (x1 == -1) and (x2 == 0):
                    x1_index = i
                elif (x1 == 0)  and (x2 == -1):
                    if len(decay["dates"]) == 0:
                        decay["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
                    else:
                        d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
                        sum = d + timedelta(seconds=1)
                        decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
        else:            
            #average decay rate
            for i in range(1,len(df) -1 ):
                x1 = df["spike/deep"][i]
                x2 = df["spike/deep"][i+1]
                if (x1 == 1) and (x2 == 0):
                    x1_index = i
                elif (x1 == 0)  and (x2 == 1):
                    if len(decay["dates"]) == 0:
                        decay["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
                    else:
                        d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
                        sum = d + timedelta(seconds=1)
                        decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))

    except UnboundLocalError as e:
        print("Error: Variable 'x1_index' referenced before assignment.")
        print(f"Exception: {e}")

    return decay


In [28]:
#Add one second
def add_sec(date):
    d = datetime.strptime(date, "%H:%M:%S:%f")
    return d + timedelta(seconds=1)


In [29]:
#Adding 3 points between every two points
def add_points(df,lead):
    new = {"dates" : [],
            "values" : []
        }
    for i in range(len(df)-1):
        x1 = df[i]      #left
        x2 = df[i+1]    #right
        m = (x1+x2)/2   #middle
        lm = (x1+m)/2   #left_middle
        rm = (m+x2)/2   #right_middle
        if len(new["dates"]) == 0:
            new["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            new["values"].append(x1)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)
        else:
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)

            print(new["dates"])

    #Returning the dict populated with the original points + new additional points + forecast points
    return insert_missing_rows(new,lead)

            

In [34]:
#forecast dots to the future
lead = 10
#calc the decay rate of the history
res = calculate_decay_rate(df)
res_df = pd.DataFrame(data = res)

#add the 3 points between 2 points and the lead points to the future
new = add_points(res_df["values"],lead)
new_res_df = pd.DataFrame(data = new)

#Plot the data
ax = new_res_df[5:].plot(legend=True,figsize=(16,8))
ax.set_xlim([5,len(new_res_df)-lead])

res_df = new_res_df

[]


IndexError: list index out of range

In [49]:
#Auto Arima To the dacay list
train = res_df[1:-lead]
test = res_df[-lead:]

trend = []
for i in range(lead -1):
    trend.append(lead -1 -i)

auto_arima(train["values"], exogenous = trend,trend="t").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   52
Model:               SARIMAX(2, 0, 0)   Log Likelihood                -266.514
Date:                Sun, 21 May 2023   AIC                            541.029
Time:                        15:24:50   BIC                            548.834
Sample:                             0   HQIC                           544.021
                                 - 52                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
drift          0.4484      0.515      0.870      0.384      -0.561       1.458
ar.L1          1.8823      0.132     14.292      0.000       1.624       2.140
ar.L2         -0.9073      0.180     -5.038      0.000      -1.260      -0.554
sigma2      1445.0043    284.882      5.072      0.000     886.646    2003.362
===================================================================================
Ljung-Box (L1) (Q):                   0.17   Jarque-Bera (JB):              4084.32
Prob(Q):                              0.68   Prob(JB):                         0.00
Heteroskedasticity (H):             339.84   Skew:                             6.41
Prob(H) (two-sided):                  0.00   Kurtosis:                        44.48
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [55]:
from matplotlib import pyplot

model = SARIMAX(endog=train["values"], order=(1,1,0))
model_fit = model.fit()

start = len(train["values"])
end = len(train["values"]) + lead
prediction = model_fit.predict(start, end,exog=trend).rename("prediction")

ax = res_df["values"][1:].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

"""
# summary of fit model
print(model_fit.summary())
"""
# line plot of residuals
#residuals = pd.DataFrame(model_fit.resid)
#residuals.plot()

# density plot of residuals
#residuals.plot(kind='kde')
#pyplot.show()
"""
# summary stats of residuals
print(residuals.describe())
"""

'\n# summary stats of residuals\nprint(residuals.describe())\n'

In [53]:
prediction

53    1410.062718
54    1659.105393
55    1893.982373
56    2115.499413
57    2324.416433
58    2521.450130
59    2707.276434
60    2882.532829
61    3047.820536
62    3203.706581
63    3350.725737
Name: prediction, dtype: float64